In [2]:
import sys
sys.path.append('../')

import os

import cqt
import cqt.env.mkt_env as env 
import cqt.env.mkt_env_spot as envspot
import cqt.ledger.ledger as ledger
import cqt.datagen as dg
import cqt.dbutility.dbutility as db
import cqt.strats.strategy as stg
import cqt.strats.strategy_long_short_average as stg_ls
import pandas as pd
import pickle

from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import copy
import cqt.strats.StrategyTensorFlow as stg_tf

outDir = '../performance_tracking'
if not os.path.exists(outDir):
    os.makedirs(outDir)
    print(outDir)
    
#pickleFileDir = '../data/pickle'
#file_btc = '../data/pickle/coinapi_ohlcv_GEMINI_SPOT_BTC_USD_1DAY_data.pickle'
#df_btc = pd.read_pickle(file_btc)
#file_eth = '../data/pickle/coinapi_ohlcv_GEMINI_SPOT_ETH_USD_1DAY_data.pickle'
#df_eth = pd.read_pickle(file_eth)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
TODAY = datetime.now()
BACK_TEST_START_DATE = datetime(2017,1,1)


In [4]:
tbl_name='coinapi-ohlcv-COINBASE_SPOT_BTC_USD-1DAY'
df_btc=db.get_from_db(tbl_name,from_date='2016-01-01',to_date=TODAY.strftime('%Y-%M-%d'))


In [9]:
df_btc.data

,price_close,price_open,price_high,price_low,time_close,time_open,trades_count,volume_traded,key,last_updated
0,435.66,430.35,437.15,427.92,2016-01-01T23:59:01.0000000Z,2016-01-01T00:00:16.0000000Z,12169,3863.277451,2016-01-01,2018-07-02 11:51:28.312307
1,435.40,435.67,437.56,432.41,2016-01-02T23:59:23.1200000Z,2016-01-02T00:00:02.0000000Z,11493,3275.441801,2016-01-02,2018-07-02 11:51:28.312307
2,431.91,435.40,435.75,425.02,2016-01-03T23:59:59.8000000Z,2016-01-03T00:00:37.7930000Z,14239,3904.335318,2016-01-03,2018-07-02 11:51:28.312307
3,433.85,431.90,435.79,431.37,2016-01-04T23:59:54.9330000Z,2016-01-04T00:00:34.1730000Z,11331,5893.789223,2016-01-04,2018-07-02 11:51:28.312307
4,433.34,433.84,435.64,430.00,2016-01-05T23:59:31.3770000Z,2016-01-05T00:00:28.7270000Z,13848,5150.109476,2016-01-05,2018-07-02 11:51:28.312307
5,430.87,433.32,433.46,428.15,2016-01-06T23:59:55.6670000Z,2016-01-06T00:00:01.6200000Z,15951,5476.959959,2016-01-06,2018-07-02 11:51:28.312307
6,459.07,430.66,460.15,430.64,2016-01-07T23:59:59.0570000Z,2016-01-07T00:00:02.2870000Z,29221,13907.201729,2016-01-07,2018-07-02 11:51:28.312307
7,454.44,459.07,464.40,447.53,2016-01-08T23:59:32.7770000Z,2016-01-08T00:00:02.2970000Z,20050,8341.783440,2016-01-08,2018-07-02 11:51:28.312307
8,450.38,454.41,456.00,447.66,2016-01-09T23:59:10.8700000Z,2016-01-09T00:00:12.5570000Z,18769,4241.356651,2016-01-09,2018-07-02 11:51:28.312307
9,449.99,450.39,451.39,442.96,2016-01-10T23:59:40.1370000Z,2016-01-10T00:00:01.6270000Z,11582,3954.322400,2016-01-10,2018-07-02 11:51:28.312307


In [5]:
config = {'asset_type': 'spot'}
model_btc_eth = env.MktEnv([])
comp_btc = envspot.MktEnvSpot('btc', df_btc.get_index_data(), config)
model_btc_eth.insert_section(comp_btc)



cash = 10000
assets = {'btc': 0}
ini_pfo = ledger.Ledger(assets, cash)


ls_rule = {'method' : 'moving_average', 'window_size' : [3, 5], 'tolerance_up' : 0.03, 'tolerance_down' : 0.03}



In [22]:
df_btc.data

,time_close,time_open,trades_count,price_low,price_open,price_close,key,volume_traded,price_high,last_updated
0,2017-12-18T23:59:59.1860000Z,2017-12-18T20:45:59.3960000Z,7229,18465.39,18493.04,18856.25,2017-12-18,704.460530,18900.00,2018-07-12 02:21:12.671337
1,2017-12-19T23:59:59.6890000Z,2017-12-19T00:00:00.0730000Z,116208,16300.00,18856.25,17295.20,2017-12-19,13167.116874,18950.00,2018-07-12 02:21:12.671337
2,2017-12-20T23:59:59.8780000Z,2017-12-20T00:00:00.3950000Z,125183,14800.00,17295.20,16488.98,2017-12-20,13008.779850,17720.35,2018-07-12 02:21:12.671337
3,2017-12-21T23:59:58.3940000Z,2017-12-21T00:00:00.1850000Z,115327,14100.00,16480.52,15492.64,2017-12-21,19897.862912,17309.50,2018-07-12 02:21:12.671337
4,2017-12-22T23:59:59.4950000Z,2017-12-22T00:00:03.9480000Z,224659,11111.51,15514.03,13326.61,2017-12-22,33064.896519,15699.34,2018-07-12 02:21:12.671337
5,2017-12-23T23:59:59.8580000Z,2017-12-23T00:00:00.2170000Z,131177,12978.18,13326.61,13300.00,2017-12-23,12808.525912,14950.00,2018-07-12 02:21:12.671337
6,2017-12-24T23:59:59.8400000Z,2017-12-24T00:00:01.3090000Z,164401,11798.80,13300.00,13500.00,2017-12-24,27079.449803,13819.99,2018-07-12 02:21:12.671337
7,2017-12-25T23:59:54.4220000Z,2017-12-25T00:00:00.4210000Z,115415,12708.00,13500.00,13699.34,2017-12-25,15607.598265,14300.00,2018-07-12 02:21:12.671337
8,2017-12-26T23:59:59.2620000Z,2017-12-26T00:00:00.9930000Z,131560,13533.00,13699.34,15689.01,2017-12-26,14845.640226,16050.00,2018-07-12 02:21:12.671337
9,2017-12-27T23:59:56.1500000Z,2017-12-27T00:00:04.5120000Z,142041,14200.15,15709.98,15459.99,2017-12-27,14936.933071,16498.05,2018-07-12 02:21:12.671337


In [16]:
strats = stg_tf.StrategyTensorFlow(model_btc_eth, ini_pfo, ls_rule, 'model.json', 'model.h5', 'scaler.pkl')

report = strats.back_testing(BACK_TEST_START_DATE)


OSError: Unable to open file (file signature not found)

In [6]:
strats_train = stg_tf.StrategyTensorFlow(model_btc_eth, ini_pfo, ls_rule)

In [ ]:
strats_train.train()

../cqt/strats/StrategyTensorFlow.py:151: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, kernel_initializer="lecun_uniform")`
  model.add(Dense(4, init='lecun_uniform'))
../cqt/strats/StrategyTensorFlow.py:243: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  model.fit(X_train, y_train, batch_size=batchSize, nb_epoch=1, verbose=0)


debug final 2410.67028295895
Epoch #: 0 Reward: 2410.670283 Epsilon: 1.000000
Saved model to disk
debug final 22882.203212432833
Epoch #: 1 Reward: 22882.203212 Epsilon: 0.900000
Saved model to disk
debug final 1458.6003548223619
Epoch #: 2 Reward: 1458.600355 Epsilon: 0.800000
Saved model to disk
debug final 528546.5574259362
Epoch #: 3 Reward: 528546.557426 Epsilon: 0.700000
Saved model to disk
debug final 597.246614968576
Epoch #: 4 Reward: 597.246615 Epsilon: 0.600000
Saved model to disk
debug final 8073.1915951520605
Epoch #: 5 Reward: 8073.191595 Epsilon: 0.500000
Saved model to disk
debug final 9591.742322970736
Epoch #: 6 Reward: 9591.742323 Epsilon: 0.400000
Saved model to disk
